# load

In [6]:
import torch
import torch.optim as optim

import torch.nn as nn
import torch.nn.functional as F



class CNNPointDetector(nn.Module):
    def __init__(self):
        super(CNNPointDetector, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # Output (x, y) coordinates
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, 64 * 8 * 8)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model
model = CNNPointDetector()
# 初始化模型
model = CNNPointDetector()
model.load_state_dict(torch.load('fan_corner2_detector_1127.pth'))  # 加载模型参数
model.train()

C:\Users\raykuo7\AppData\Local\Temp\ipykernel_7592\3076898848.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('fan_corner2_detector_112

CNNPointDetector(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

# newstart

In [1]:
import pandas as pd
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class FanDataset(Dataset):
    def __init__(self, annotations_file, img_size=64):
        self.annotations = pd.read_csv(annotations_file)
        self.img_size = img_size
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_path = self.annotations.iloc[idx, 0]
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (self.img_size, self.img_size)) / 2047.0  # Resize and normalize
        image = image.reshape(1, self.img_size, self.img_size)  # Reshape for CNN input

        # Label: x and y coordinates of the corner
        label = self.annotations.iloc[idx, 1:3].values.astype(np.float32)
        return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# 使用自己的标注文件
dataset = FanDataset('selected_points2_1127.csv')
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class CNNPointDetector(nn.Module):
    def __init__(self):
        super(CNNPointDetector, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # Output (x, y) coordinates
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, 64 * 8 * 8)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model
model = CNNPointDetector()

In [3]:
import torch.optim as optim

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 400
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}", 'about',(num_epochs-epoch)/1,'min left')

Epoch [1/400], Loss: 5623278.1000 about 400.0 min left
Epoch [2/400], Loss: 5611610.0000 about 399.0 min left
Epoch [3/400], Loss: 5554933.9000 about 398.0 min left
Epoch [4/400], Loss: 5371646.0000 about 397.0 min left
Epoch [5/400], Loss: 4893590.0000 about 396.0 min left
Epoch [6/400], Loss: 3879892.9500 about 395.0 min left
Epoch [7/400], Loss: 2157196.5500 about 394.0 min left
Epoch [8/400], Loss: 454533.3875 about 393.0 min left
Epoch [9/400], Loss: 635091.3375 about 392.0 min left
Epoch [10/400], Loss: 384094.4469 about 391.0 min left
Epoch [11/400], Loss: 210146.6656 about 390.0 min left
Epoch [12/400], Loss: 296199.1219 about 389.0 min left
Epoch [13/400], Loss: 203842.8031 about 388.0 min left
Epoch [14/400], Loss: 178479.7094 about 387.0 min left
Epoch [15/400], Loss: 187817.8953 about 386.0 min left
Epoch [16/400], Loss: 159926.7812 about 385.0 min left
Epoch [17/400], Loss: 165918.8094 about 384.0 min left
Epoch [18/400], Loss: 165021.6094 about 383.0 min left
Epoch [19/40

In [4]:
torch.save(model.state_dict(), './fan_corner2_detector_1127.pth')

# keep training

In [7]:
import pandas as pd
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class FanDataset(Dataset):
    def __init__(self, annotations_file, img_size=64):
        self.annotations = pd.read_csv(annotations_file)
        self.img_size = img_size
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_path = self.annotations.iloc[idx, 0]
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (self.img_size, self.img_size)) / 2047.0  # Resize and normalize
        image = image.reshape(1, self.img_size, self.img_size)  # Reshape for CNN input

        # Label: x and y coordinates of the corner
        label = self.annotations.iloc[idx, 1:3].values.astype(np.float32)
        return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# 使用自己的标注文件
dataset = FanDataset('selected_points2_1127.csv')
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [8]:
import torch.optim as optim

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 420
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}", 'about',(num_epochs-epoch)/1,'min left')

Epoch [1/420], Loss: 80679.5219 about 420.0 min left
Epoch [2/420], Loss: 74710.0625 about 419.0 min left
Epoch [3/420], Loss: 69001.6891 about 418.0 min left
Epoch [4/420], Loss: 69036.4406 about 417.0 min left
Epoch [5/420], Loss: 70073.2281 about 416.0 min left
Epoch [6/420], Loss: 69226.7734 about 415.0 min left
Epoch [7/420], Loss: 67935.0734 about 414.0 min left
Epoch [8/420], Loss: 67592.1789 about 413.0 min left
Epoch [9/420], Loss: 67080.5969 about 412.0 min left
Epoch [10/420], Loss: 67303.3047 about 411.0 min left
Epoch [11/420], Loss: 67172.6711 about 410.0 min left
Epoch [12/420], Loss: 66800.2422 about 409.0 min left
Epoch [13/420], Loss: 67115.7766 about 408.0 min left
Epoch [14/420], Loss: 66319.8547 about 407.0 min left
Epoch [15/420], Loss: 66755.3766 about 406.0 min left
Epoch [16/420], Loss: 66477.3961 about 405.0 min left
Epoch [17/420], Loss: 67769.2797 about 404.0 min left
Epoch [18/420], Loss: 67402.3008 about 403.0 min left
Epoch [19/420], Loss: 66402.6070 abou

In [9]:
torch.save(model.state_dict(), './fan_corner2_detector_1128.pth')